In [1]:
import dace
import cupy as cp

In [2]:
N = dace.symbol('N')
M = dace.symbol('M')
sN = 33
sM = 21

@dace.program
def vector_copy_strides2d(A: dace.uint32[N, M] @ dace.dtypes.StorageType.GPU_Global, B: dace.uint32[N, M] @ dace.dtypes.StorageType.GPU_Global):
    for i, j in dace.map[0:N:sN, 0:M:sM] @ dace.dtypes.ScheduleType.GPU_Device:
        A[i, j] = B[i, j]

sdfg = vector_copy_strides2d.to_sdfg()
sdfg

SDFG (vector_copy_strides2d)

In [3]:
# Example input 
n = 35
m = 43
A = cp.zeros((n, m,), dtype=cp.uint32)
B = cp.ones((n, m,), dtype=cp.uint32)


# Strided copy from B to A
sdfg(A=A, B=B, N=n, M=m)


# Verify correctness for 2D strided copy
expected = cp.zeros((n, m), dtype=cp.uint32)
expected[::sN, ::sM] = cp.ones((n, m,), dtype=cp.uint32)[::sN, ::sM]
if (A == expected).all():
    print("\n\nSUCCESS: A matches the expected result.")
else:
    print("\n\nERROR: A does not match the expected result.")



SUCCESS: A matches the expected result.
